<a href="https://colab.research.google.com/github/cowiety/energy-forecast/blob/main/LSTM_dataset/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Model

import statements

In [111]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Reshape
from sklearn.model_selection import train_test_split
import pandas as pd

open data files and convert to numpy arrays

In [80]:
# open uploaded csv file
demand_file = open('Demand Data (2021-12-03).csv')
# generate numpy array from file
demand = np.genfromtxt(demand_file, delimiter=',', skip_header=1, usecols=(1))

weather_file = open('Historical Weather Data (2021-12-03).csv')
weather = np.genfromtxt(weather_file, delimiter=',', skip_header=1, usecols=(1,2,3,4))

holiday_file = open('isHoliday (2021-12-03).csv')
holiday = np.genfromtxt(holiday_file, delimiter=',', skip_header=1, usecols=(1))

function for normalizing the weather data

In [75]:
def normalize(data):
  data_mean = data[:].mean(axis=0)
  data_std = data[:].std(axis=0)
  return (data-data_mean) / data_std

split the 2D weather array into 4 x 1D arrays

In [81]:
# apparently there's an empty row at the bottom that drove me insane for 2 hours
weather = weather[:-1]

#normalize weather and demand data
weather = normalize(weather)
demand = demand / 1000

# splitting weather array into a dictionary called weather_comp
weather_comp = np.split(weather, 4, axis=1)
# putting each component into its own variable
clouds = weather_comp[0]
temp = weather_comp[1]
humidity = weather_comp[2]
wind = weather_comp[3]

combine all the features into one array

In [89]:
data = np.column_stack((demand, holiday, clouds, temp, humidity, wind))
np.savetxt("lstm_full.csv", data, delimiter=",")
print(data)

[[19.436       0.          0.3910058  -0.89400393  1.32692069 -0.18638073]
 [19.734       0.          0.41204612 -0.82034213  1.30610063  0.12258661]
 [19.994       0.          1.15273137 -0.76485661  1.23228405 -0.26300855]
 ...
 [19.936       0.          0.47385208 -0.89878717 -0.74940707 -0.99686737]
 [19.384       0.          0.57017733 -0.92844322 -0.64341404 -0.99686737]
 [18.206       0.          0.57017733 -0.97531892 -0.46928264 -0.99686737]]


reshape into a 3D array to prepare the data for LSTM model

In [105]:
y = data[:, 0]
x = data[:, 1:]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False)
#print(X_train.shape)

In [123]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

x_len = x_train.shape[1]
n_comps = int(x_len / 5)
input_shape = (x_len, 1)
units = 32

In [125]:
print(n_comps)

1


creating a RNN with one hidden LSTM layer and one Dense output layer

In [132]:
model = Sequential()
model.add(Reshape((5, n_comps), input_shape=input_shape))
model.add(LSTM(units, input_shape=input_shape))
model.add(Dense(1, activation='softmax'))
model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_10 (Reshape)        (None, 5, 1)              0         
                                                                 
 lstm_22 (LSTM)              (None, 32)                4352      
                                                                 
 dense_21 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [133]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
model.compile(optimizer=opt, loss = 'mse', metrics=['accuracy'])
history = model.fit(
    np.array(x_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 25,
    validation_split = 0.10
)

Epoch 1/25
158/158 [==============================] - 6s 14ms/step - loss: 270.7181 - accuracy: 0.0000e+00 - val_loss: 284.4273 - val_accuracy: 0.0000e+00
Epoch 2/25
158/158 [==============================] - 1s 6ms/step - loss: 270.7182 - accuracy: 0.0000e+00 - val_loss: 284.4273 - val_accuracy: 0.0000e+00
Epoch 3/25
158/158 [==============================] - 1s 6ms/step - loss: 270.7181 - accuracy: 0.0000e+00 - val_loss: 284.4273 - val_accuracy: 0.0000e+00
Epoch 4/25
158/158 [==============================] - 1s 6ms/step - loss: 270.7182 - accuracy: 0.0000e+00 - val_loss: 284.4273 - val_accuracy: 0.0000e+00
Epoch 5/25
158/158 [==============================] - 1s 6ms/step - loss: 270.7181 - accuracy: 0.0000e+00 - val_loss: 284.4273 - val_accuracy: 0.0000e+00
Epoch 6/25
158/158 [==============================] - 1s 6ms/step - loss: 270.7182 - accuracy: 0.0000e+00 - val_loss: 284.4273 - val_accuracy: 0.0000e+00
Epoch 7/25
158/158 [==============================] - 1s 6ms/step - loss: 2